In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os, sys
import csv
import string

import psycopg2
import pandas as pd

from sklearn.cluster import KMeans

In [2]:
# need tensorflow, tensorflow_hub, numpy, psycopg2, pandas, and sklearn
# All of these packages should be downloadable with pip

# connect to the PostgreSQL database server
def connect(params_dic):
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [3]:
# convert postgresql db to pandas df
def postgresql_to_dataframe(conn, select_query, column_names):
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

In [4]:
# Connection parameters
param_dic = {
    "host"      : "ec2-3-35-78-167.ap-northeast-2.compute.amazonaws.com",
    "database"  : "postgres",
    "user"      : "postgres",
    "password"  : "postgres"
}

In [5]:
# Connect to the database
conn = connect(param_dic)

df = postgresql_to_dataframe(conn, "select text from fb_mychatbot_chat where said_by='USR'", ['text'])
df = df['text']
df = df.astype('str')
np_messages = df.to_numpy()

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embed = hub.load(module_url)
prelim_sent_list = []
freq_list = []
sentences_dict = {}
sentences_list = []

Connecting to the PostgreSQL database...
Connection successful


In [6]:
# Lowercasing all of the responses and recording number of times each response came up
for i in np_messages:
    i = i.lower()
    i = i.translate(str.maketrans('', '', string.punctuation))
    if i.isspace():
        continue
    if len(i) == 0:
        continue
    if i in sentences_dict:
        sentences_dict[i] = sentences_dict[i] + 1
    else:
        sentences_dict[i] = 1
        prelim_sent_list.append(i)

In [7]:
# This is supposed to be used to get rid of all of the responses that are of a lower frequency.

# Finds the frequency of all of the sentences
for key in sentences_dict:
    freq_list.append(sentences_dict[key])

npfreq = np.array(freq_list)

cutoff_freq = np.percentile(npfreq, 95)

count = 0
for sent in prelim_sent_list:
    if sentences_dict[sent] == 1:
        count += 1
print("the number of messages with freq 1 is "+str(count))

the number of messages with freq 1 is 14638


In [8]:
# Only adds sentences that are in the top 5% most frequent responses
for sent in prelim_sent_list:
    if sentences_dict[sent] > cutoff_freq:
        sentences_list.append(sent)

In [9]:
# USE at work
embeddings = embed(sentences_list)

In [11]:
# Finding a more granular cluster_cnt
cluster_cnt = 1
for i in range(len(sentences_list)):
    granular = len(sentences_list) / cluster_cnt
    if granular <= 1.6:
        break
    else:
        cluster_cnt += 2
        
CLUSTER_CNT = int(cluster_cnt)

In [12]:
# Clustering the data that went through the USE
kmeans = KMeans(n_clusters = CLUSTER_CNT)
kmeans.fit(embeddings)

labels = kmeans.labels_
labelDict = dict()

In [13]:
# Inputting the clustered data into a dictionary
for i, sentence in enumerate(sentences_list):
    if labels[i] in labelDict.keys():
        labelDict[labels[i]].append(sentence)
    else:
        labelDict[labels[i]] = [sentence]

In [14]:
# Reordering the cluster numberings to output the clusters from 0 to CLUSTER_CNT - 1
cluster_id = []
for label in labelDict.keys():
    cluster_id.append(label)
cluster_id.sort()

In [15]:
# writing out the information into a CSV file for reading by typist
# If you have provided a statement, then that specific cluster will be outputted

with open('fbmchat_output.csv', mode='w') as csv_file:
    writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['cluster', 'frequency', 'message'])
    for label in cluster_id:
        for sentence in labelDict[label]:
            writer.writerow([label, sentences_dict[sentence], sentence])